# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

import yaml


In [2]:
from ipywidgets import *

In [3]:
start_dir=os.getcwd()
print(start_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/run_scripts


## Define machine and code 

In [4]:
# facility='cori'
# img_size=64
# run_type='3dgan'
# assert facility in ['cori','summit'], "Error%s"%(facility)
# assert img_size in [64,128,512]
# assert run_type in ['2dgan','2dcgan','3dgan','3dcgan']

In [5]:
def f(facility,img_size,run_type): return facility,img_size,run_type
w=interactive(f,
                facility=ToggleButtons(options=['cori','summit']),
                img_size=ToggleButtons(options=[64,128,512]),
                run_type=ToggleButtons(options=['2dgan','2dcgan','3dgan','3dcgan']) )
display(w)


interactive(children=(ToggleButtons(description='facility', options=('cori', 'summit'), value='cori'), ToggleB…

In [7]:
result=w.result
facility,img_size,run_type=result
print(facility,img_size,run_type)

cori 64 3dgan


## Read ref_launch.yaml and define dictionary

In [8]:
# start_dir='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/run_scripts'
launch_cfile=start_dir+'/ref_launch.yaml'

with open(launch_cfile) as f:
    config_dict= yaml.load(f, Loader=yaml.SafeLoader)

In [9]:
## Read ref_launch.yaml
main_code_dir=config_dict[facility]['main_code_dir']
dict_pars=config_dict[facility][img_size][run_type]
dict_pars.update({'nodes':1,'job_name':'analysis_'+run_type,
                 'queue':'debug','bins':'unenven','cores':64,'time':'00:30:00'})

dict_pars['code_dir']=main_code_dir+dict_pars['code_dir']
dict_pars['staging_loc']=config_dict[facility]['staging_loc']
dict_pars['img_size']=img_size
print(dict_pars)

staging_loc=dict_pars['staging_loc']
op_fldr=dict_pars['staging_loc']+dict_pars['op_dir']
op_fldr

{'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/4_basic_3d_GAN/1_main_code/DDP', 'op_dir': '3d/', 'config': 'config_3dgan.yaml', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy', 'nodes': 1, 'job_name': 'analysis_3dgan', 'queue': 'debug', 'bins': 'unenven', 'cores': 64, 'time': '00:30:00', 'staging_loc': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/', 'img_size': 64}


'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/'

In [10]:
# %%bash -s "$op_fldr"
# ls $1

In [11]:
# parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(op_fldr+'202105*')]
w=interactive(lambda x: x, x=ToggleButtons(options=dir_lst))
display(w)

interactive(children=(ToggleButtons(description='x', options=('20210514_120637_128cube_bs4_lr0.002_nodes16', '…

In [13]:
# ip_folder='20210507_121514_cgan_bs8_lr0.00005_nodes8'
ip_folder=w.result
dict_pars['ip_folder']=dict_pars['staging_loc']+dict_pars['op_dir']+ip_folder
print(dict_pars['ip_folder'])

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210518_91712_bs8_lr0.001_nodes8_spec0.1


In [14]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir={code_dir}

python $code_dir/../post_analysis_pandas.py -f {ip_folder}  -v {val_file} -c {cores} -bin {bins} -i {img_size}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [15]:
fname='batch_analysis_{0}.sh'.format(run_type)
filename=dict_pars['staging_loc']+fname
with open (filename,'w') as f:
    f.write(bash_strg)
print(filename)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_analysis_3dgan.sh


In [16]:
## Move to staging locations in project space:
os.chdir(dict_pars['staging_loc'])

In [17]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:30:00
#SBATCH --job-name=analysis_3dgan

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/4_basic_3d_GAN/1_main_code/DDP

python $code_dir/../post_analysis_pandas.py -f /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210518_91712_bs8_lr0.001_nodes8_spec0.1  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy -c 64 -bin unenven -i 64
conda deactivate
echo "--end date" `date` `date +%s`


## Submit job to cori GPU

In [18]:
%%bash -s "$filename" ## Use python variable in bash
# module load cgpu
sbatch $1

Submitted batch job 42717645


In [ ]:
pwd